## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load necessary data
future_data = pd.read_excel('../data/fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name=0)
bond_data = pd.read_excel('../data/fut_bond_data_FVM5_2025-02-25.xlsx', sheet_name=1)

### Data Handling & Global Variables

In [3]:
# Futures Price (F)
F = future_data.loc[future_data['field'] == 'px_last', 'FVM5 Comdty'].iloc[0]
F = float(F)

# Extract current quote date and delivery date
t_current = pd.to_datetime(future_data.loc[future_data['field'] == 'last_update_dt', 'FVM5 Comdty'].iloc[0])
T_delivery = pd.to_datetime(future_data.loc[future_data['field'] == 'last_tradeable_dt', 'FVM5 Comdty'].iloc[0])

# Time to Delivery (tau) in ACT/360
days_to_delivery = (T_delivery - t_current).days
tau = days_to_delivery / 360.0

# Display the global variables
display(pd.DataFrame({
    'Variable': ['Futures Price (F)', 'Days to Delivery', 'Tau (ACT/360)'],
    'Value': [F, days_to_delivery, round(tau, 6)]
}))

,Variable,Value
0,Futures Price (F),107.406250
1,Days to Delivery,124.000000
2,Tau (ACT/360),0.344444


## 1. Cheapest-to-Deliver Analysis

### 1.1 Delivery Cost

The **delivery cost** measures the financial cost to the short position to buy the bond and deliver it against the futures contract. It is defined as:

$$
\text{Delivery Cost}^i = P_t^i - \phi^i \times F(t,T)
$$

Where:

- $P_t^i$ : Clean price of the $i$-th bond  
- $\phi^i$ : Conversion factor for the $i$-th bond  
- $F(t,T)$ : Futures price  

The bond with the lowest delivery cost is called the **Cheapest to Deliver (CTD)**.

In [4]:
# Calculate Delivery Cost (Formula: P - (Conversion Factor * Futures Price))
bond_data['delivery_cost'] = bond_data['px_last'] - (bond_data['conversion'] * F)

# Identify the bond with the minimum delivery cost
min_cost_idx = bond_data['delivery_cost'].idxmin()
ctd_candidate_cost = bond_data.loc[min_cost_idx]

# Display the bond with the lowest delivery cost
display(pd.DataFrame({
    'Ticker': [ctd_candidate_cost['ticker']],
    'Delivery Cost': [round(ctd_candidate_cost['delivery_cost'], 6)]
}))

,Ticker,Delivery Cost
0,91282CLK Govt,-0.185922


### 1.2 Identify the CTD

The **Cheapest-to-Deliver (CTD)** bond is formally identified as the one with the lowest Delivery Price Ratio (or, similarly, the lowest basis). The delivery price ratio indicates the cost per unit of the conversion factor:

$$
\text{Delivery Price Ratio}^i = \frac{P_t^i}{\phi^i}
$$

where:
- $P_t^i$ is the clean price of the $i$-th bond  
- $\phi^i$ is the conversion factor of the $i$-th bond

We calculate this ratio for all bonds to confirm the CTD identity. We then compare our result to the `fut_ctd` field provided in the `future_data`.

In [5]:
# Calculate Delivery Price Ratio
bond_data['delivery_price_ratio'] = bond_data['px_last'] / bond_data['conversion']

# Identify CTD based on minimum ratio
ctd_idx = bond_data['delivery_price_ratio'].idxmin()
ctd_bond = bond_data.loc[ctd_idx]

# Display CTD Ticker and Delivery Price Ratio as a DataFrame
display(
    pd.DataFrame({
        'CTD Ticker': [ctd_bond['ticker']],
        'Delivery Price Ratio': [round(ctd_bond['delivery_price_ratio'], 6)]
    })
)

# Compare with Bloomberg Field
bbg_ctd = future_data.loc[future_data['field'] == 'fut_ctd', 'FVM5 Comdty'].iloc[0]
display(pd.DataFrame({'Bloomberg Reported CTD': [bbg_ctd]}))

,CTD Ticker,Delivery Price Ratio
0,91282CLK Govt,107.202723


,Bloomberg Reported CTD
0,T 3.625 08/31/29


**Question : Does this match the CTD identified by Bloomberg in the fut_ctd field?**

Answer: Yes, this matches the CTD identified by Bloomberg in the fut_ctd field, as 91282CLK Govt matures on 08/31/29 and has a 3.625% coupon—the same as the bond listed as CTD by Bloomberg.

### 1.3 CTD Intuition

#### Coupons and Maturities of CTD Candidates

The bonds with the lowest Delivery Price Ratios are currently the "low coupon" bonds in the 4–5 year maturity range. Specifically, based on the provided data, the CTD is the **T 3.625 08/31/29**. This bond has a lower coupon compared to others in the list (e.g., 4.125% or 4.375%).

#### High vs. Low Coupon in the Current Environment

- **Environment:** Rates are approximately 4–5%.
- **Expectation:** When yields are below 6%, **high-coupon (lower-duration) bonds** tend to be CTD (all else equal, ignoring repo specialness and liquidity differences).

**Reasoning:**  
Treasury futures conversion factors are computed under a **flat 6% yield** assumption. When market yields are **below 6%**, the conversion-factor system effectively makes **low-coupon (high-duration) bonds look expensive to deliver** and **high-coupon (low-duration) bonds look cheap to deliver**. Intuitively, the futures contract is “standardized” around a 6% bond; in a <6% world, delivering a **long-duration/low-coupon** bond gives the long **more duration than the futures price is designed to pay for**, which is unfavorable for the short and therefore not CTD. Conversely, delivering a **shorter-duration/high-coupon** bond better matches (or under-delivers) duration relative to the 6% standardization, reducing the delivery cost and tending to minimize the basis / maximize implied repo, making it CTD. Repo specialness can override this ranking, but directionally the <6% environment favors **higher coupons** for CTD.


#### Why CBOT Uses 6%

The 6% notional rate is a historical convention. It was originally chosen to represent a "typical" market yield to standardize the contract. While it creates a bias when rates deviate significantly from 6%, it remains the standard to maintain contract continuity and liquidity.

## 2. Implied Repo Rate

### 2.1 Calculate Implied Repo

The **Implied Repo Rate** ($r_{IR}$) is the rate of return a trader earns by buying the bond in the cash market, shorting the future, and delivering the bond into the futures contract.

The formula for the Implied Repo Rate is:

$$
r_{IR}^i = \frac{1}{\tau_{act/360}} \left[ \frac{F(t,T) \cdot \phi^i + A_T^i}{P_t^i + A_t^i} - 1 \right]
$$

Where:
- $F(t,T)$ = Futures price at time $t$ for delivery at $T$
- $\phi^i$ = Conversion factor for bond $i$
- $A_T^i$ = Accrued interest on bond $i$ at delivery
- $P_t^i$ = Cash price of bond $i$ at time $t$
- $A_t^i$ = Accrued interest on bond $i$ at time $t$
- $\tau_{act/360}$ = Actual/360 year fraction from now until delivery

**Calculation of Accrued Interest at Delivery ($A_T$):**  
We must project the accrued interest to the delivery date:

- Calculate days accrued at delivery:  
  $$
  \text{accrued at delivery} = \text{days\_acc\_current} + \text{days\_to\_delivery}
  $$
- If a coupon payment occurs between now and delivery, the accrual resets at the coupon date.

**Note on Data:**  
The `nxt_cpn_dt` column helps us detect if a coupon falls within the holding period. We approximate $A_T$ using the provided coupon and day count data.

In [6]:
# Function to calculate Accrued Interest at Delivery (A_T)
def calculate_A_T(row, days_to_dlv):
    """
    Calculates Accrued Interest at Delivery (A_T) handling potential coupon payments.
    """
    # Check if a coupon falls between now and delivery
    if days_to_dlv >= row['days_to_next_coupon']:
        # Coupon paid during holding period
        # Days accrued at delivery = Days from (Next Coupon) to (Delivery)
        days_acc_at_T = days_to_dlv - row['days_to_next_coupon']
    else:
        # No coupon paid
        # Days accrued at delivery = Current Accrued + Days to Delivery
        days_acc_at_T = row['days_acc'] + days_to_dlv
    
    # Calculate A_T amount
    # Formula: (Coupon / 2) * (Days Accrued / Days in Period)
    A_T = (row['cpn'] / 2) * (days_acc_at_T / row['accrued_days_between_cpn_dates'])
    return A_T

# Calculate A_T for all bonds
bond_data['acc_int_delivery'] = bond_data.apply(lambda row: calculate_A_T(row, days_to_delivery), axis=1)

# Calculate Implied Repo Rate
# Formula: r_IR = (1/tau) * [(F * phi + A_T) / (P + A_t) - 1]
numerator = (F * bond_data['conversion']) + bond_data['acc_int_delivery']
denominator = bond_data['px_last'] + bond_data['int_acc']
bond_data['implied_repo'] = (1 / tau) * ((numerator / denominator) - 1)

# Display Top 5 Highest Implied Repo Bonds
display_cols_ir = ['ticker', 'px_last', 'conversion', 'int_acc', 'acc_int_delivery', 'implied_repo']
print("Top 5 Bonds by Implied Repo Rate:")
display(bond_data[display_cols_ir].sort_values('implied_repo', ascending=False).head())

Top 5 Bonds by Implied Repo Rate:


,ticker,px_last,conversion,int_acc,acc_int_delivery,implied_repo
5,91282CMG Govt,100.468750,0.9307,0.316989,1.772790,0.027367
0,91282CLK Govt,97.929688,0.9135,1.802486,1.231699,-0.011204
1,91282CLN Govt,97.343750,0.9074,1.442308,0.884615,-0.012961
2,91282CLR Govt,99.937500,0.9293,1.356008,0.706492,-0.022195
3,91282CMA Govt,99.960938,0.9281,1.008585,0.351305,-0.026869


### 2.2 Compare to Actual Repo

Here we compare the calculated $r_{IR}$ (implied repo rate) against the market repo rate (`repo_reporate`) provided in the data.

In [14]:
# Convert repo rate to decimal (assuming data is in %)
bond_data['repo_reporate_val'] = bond_data['repo_reporate'].astype(float) / 100.0

# Calculate Repo Spread
bond_data['repo_spread'] = bond_data['implied_repo'] - bond_data['repo_reporate_val']

# Identify CTD and Highest Implied Repo bonds
ctd_ticker = bond_data.loc[bond_data['delivery_price_ratio'].idxmin(), 'ticker']
highest_ir_ticker = bond_data.loc[bond_data['implied_repo'].idxmax(), 'ticker']

# Get rows for analysis
ctd_row = bond_data.loc[bond_data['ticker'] == ctd_ticker].iloc[0]
highest_ir_row = bond_data.loc[bond_data['ticker'] == highest_ir_ticker].iloc[0]

# Display Comparison
comparison_df = pd.DataFrame({
    'Metric': ['Ticker', 'Implied Repo', 'Actual Repo', 'Repo Spread'],
    'CTD Bond': [
        ctd_ticker, 
        f"{ctd_row['implied_repo']:.4%}", 
        f"{ctd_row['repo_reporate_val']:.4%}", 
        f"{ctd_row['repo_spread']:.4%}"
    ],
    'Highest IR Bond': [
        highest_ir_ticker, 
        f"{highest_ir_row['implied_repo']:.4%}", 
        f"{highest_ir_row['repo_reporate_val']:.4%}", 
        f"{highest_ir_row['repo_spread']:.4%}"
    ]
})

display(comparison_df)

,Metric,CTD Bond,Highest IR Bond
0,Ticker,91282CLK Govt,91282CMG Govt
1,Implied Repo,-1.1204%,2.7367%
2,Actual Repo,4.3700%,4.3700%
3,Repo Spread,-5.4904%,-1.6333%


### 2.3 Interpret Implied Repo

**Trading Opportunity (Implied > Actual):**
 
If $r_{IR} > r_{repo}$, a **cash-and-carry arbitrage** exists:
 
1. **Buy** the bond (funded at $r_{repo}$).
2. **Short** the futures contract.
3. **Deliver** the bond at expiry.
 
The trader earns $r_{IR}$ from the package and pays $r_{repo}$, pocketing the positive spread risk-free (ignoring transaction costs and haircuts).
  
**Why high implied repo does _not_ always mean CTD?**
A bond might have a high implied repo but not be CTD if it is "special" in the repo market (i.e., has a very low $r_{repo}$). If a specific bond is in high demand, its repo rate drops near zero. This artificially inflates the spread ($r_{IR} - r_{repo}$), but the price of the bond might also be high, making it expensive to deliver (i.e., $P/\phi$ is high).

The CTD status is determined by the **Delivery Price** ($P/\phi$) or **Net Basis**, not just the implied financing rate.

## 3. Conversion Factor Calculation


In [8]:
def calculate_conversion_factor(row, delivery_date):
    """
    Calculate CBOT conversion factor using 6% flat yield assumption.
    Formula: φ = (c/0.06) * [1 - 1/(1.03)^n] + 1/(1.03)^n
    """
    # Get maturity date
    maturity = pd.to_datetime(row['maturity'])
    
    # First day of delivery month
    first_day_delivery_month = delivery_date.replace(day=1)
    
    # Calculate months from first day of delivery month to maturity
    months_to_maturity = (maturity.year - first_day_delivery_month.year) * 12 + \
                         (maturity.month - first_day_delivery_month.month)
    
    # Round to nearest quarter (3 months)
    months_rounded = round(months_to_maturity / 3) * 3
    
    # Number of semiannual periods
    n = months_rounded / 6
    
    # Coupon rate (as decimal)
    c = row['cpn'] / 100.0
    
    # Conversion factor formula
    if n > 0:
        phi = (c / 0.06) * (1 - (1 / (1.03 ** n))) + (1 / (1.03 ** n))
    else:
        phi = 1.0  # Edge case
    
    return phi

# Calculate conversion factors
bond_data['conversion_calc'] = bond_data.apply(
    lambda row: calculate_conversion_factor(row, T_delivery), axis=1
)

# Compare with Bloomberg values
bond_data['conversion_diff'] = bond_data['conversion_calc'] - bond_data['conversion']

print("Conversion Factor Comparison:")
display(bond_data[['ticker', 'maturity', 'cpn', 'conversion', 'conversion_calc', 'conversion_diff']].head(10))

print(f"\nMean absolute difference: {bond_data['conversion_diff'].abs().mean():.6f}")
print(f"Max absolute difference: {bond_data['conversion_diff'].abs().max():.6f}")

Conversion Factor Comparison:


,ticker,maturity,cpn,conversion,conversion_calc,conversion_diff
0,91282CLK Govt,2029-08-31,3.625,0.9135,0.912057,-0.001443
1,91282CLN Govt,2029-09-30,3.500,0.9074,0.907428,0.000028
2,91282CLR Govt,2029-10-31,4.125,0.9293,0.930571,0.001271
3,91282CMA Govt,2029-11-30,4.125,0.9281,0.927005,-0.001095
4,91282CMD Govt,2029-12-31,4.375,0.9367,0.936738,0.000038
5,91282CMG Govt,2030-01-31,4.250,0.9307,0.931872,0.001172



Mean absolute difference: 0.000841
Max absolute difference: 0.001443


### 3.2 Conversion Factor Bias

The Treasury futures conversion factor (CF) system is built off a **fixed 6% yield (flat curve)** assumption. This creates a **systematic CTD bias** when the *true* market yield level differs from 6%.

#### Rates < 6%: which bonds tend to be CTD?
- **Tends to favor higher-coupon, lower-duration bonds** (all else equal).
- Intuition: the futures contract is effectively “standardized” around a 6% bond. In a **below-6%** yield environment, **low-coupon bonds carry more duration** than the 6% standardization. Delivering a **long-duration / low-coupon** bond gives the long “extra” duration relative to what the futures price is designed to pay for, which is unfavorable to the short and therefore tends *not* to be CTD.

#### Rates > 6%: which bonds tend to be CTD?
- **Tends to favor lower-coupon, longer-duration bonds** (all else equal).
- Intuition flips: when yields are **above 6%**, the 6% CF standardization tends to make the **low-coupon / higher-duration** bonds relatively cheaper to deliver versus the futures invoice economics.

#### Given current rate levels (≈ 4–5%), which bonds do we expect to be close to CTD?
- Since **4–5% < 6%**, the **rule-of-thumb** is that **higher-coupon (lower-duration) deliverables** should be **closer to CTD**.
- **Important caveat:** CTD is ultimately determined by **net basis / implied repo (delivery economics)**, not coupon alone.


## 4. Quality Option Value

### 4.1 Net Basis as Option Value

The **Net Basis** represents the value of the "Quality Option"—that is, the right for the short futures holder to choose **which bond to deliver**. It is defined as:

$$
\text{Net Basis} = \text{Basis} - \text{Carry}
$$

In the data, this is reported as `basis_mid` (gross basis) minus expected carry, or directly as `fut_ctd_net_basis` in the futures dataset.

**CTD Net Basis:**  
The CTD bond (Cheapest-to-Deliver) typically has the **lowest Net Basis**, which should be close to zero but usually slightly positive.

**Comparison:**  
All other deliverable bonds will have a **significantly higher Net Basis**. The difference between a bond's Net Basis and the CTD's Net Basis represents the **incremental cost of delivering a non-CTD bond**.

In [9]:
def calculate_net_basis(row, futures_price, days_hold):
    """
    Calculate Net Basis = Basis - Carry
    Basis = P - (φ * F)
    Carry = Coupon Income - Financing Cost
    """
    # Gross Basis
    gross_basis = row['px_last'] - (futures_price * row['conversion'])
    
    # Financing Cost (repo cost on dirty price)
    dirty_price = row['px_last'] + row['int_acc']
    financing_cost = dirty_price * row['repo_reporate_val'] * (days_hold / 360.0)
    
    # Coupon Income (if coupon is paid during holding period)
    if days_hold >= row['days_to_next_coupon']:
        coupon_income = row['cpn'] / 2.0  # Semiannual coupon per 100 face
    else:
        coupon_income = 0.0
    
    # Carry = Income - Cost
    carry = coupon_income - financing_cost
    
    # Net Basis
    net_basis = gross_basis - carry
    
    return net_basis, gross_basis, carry

# Calculate net basis for all bonds
results = bond_data.apply(lambda row: calculate_net_basis(row, F, days_to_delivery), axis=1)
bond_data['net_basis_calc'] = results.apply(lambda x: x[0])
bond_data['gross_basis_calc'] = results.apply(lambda x: x[1])
bond_data['carry_calc'] = results.apply(lambda x: x[2])

print("=== Net Basis Analysis ===\n")
print("Net Basis by Bond (sorted from lowest to highest):")
display(bond_data[['ticker', 'gross_basis_calc', 'carry_calc', 'net_basis_calc']].sort_values('net_basis_calc').head(10))

# CTD Net Basis
ctd_net_basis = bond_data.loc[bond_data['ticker'] == ctd_ticker, 'net_basis_calc'].values[0]
print(f"\n--- CTD Net Basis ---")
print(f"CTD Bond: {ctd_ticker}")
print(f"Net Basis: {ctd_net_basis:.6f}")

# Compare to other bonds
non_ctd_avg = bond_data.loc[bond_data['ticker'] != ctd_ticker, 'net_basis_calc'].mean()
print(f"\nAverage net basis of non-CTD bonds: {non_ctd_avg:.6f}")
print(f"Difference: {non_ctd_avg - ctd_net_basis:.6f}")

=== Net Basis Analysis ===

Net Basis by Bond (sorted from lowest to highest):


,ticker,gross_basis_calc,carry_calc,net_basis_calc
0,91282CLK Govt,-0.185922,0.311309,-0.497231
2,91282CLR Govt,0.124872,0.537808,-0.412936
1,91282CLN Govt,-0.116681,0.263050,-0.379732
3,91282CMA Govt,0.277197,0.542684,-0.265487
4,91282CMD Govt,0.400378,0.656557,-0.256179
5,91282CMG Govt,0.505753,-1.517049,2.022802



--- CTD Net Basis ---
CTD Bond: 91282CLK Govt
Net Basis: -0.497231

Average net basis of non-CTD bonds: 0.141694
Difference: 0.638925


### 4.2 Option Interpretation

**Positive** CTD Net Basis implies that the **short position holds valuable options**. The market prices the futures contract **lower than the pure cash-and-carry fair value** to compensate the long position for the risk that the short will exercise these options:

- **Quality Option:** The ability to switch to a cheaper bond for delivery if rates change.
- **Timing Option:** The flexibility to deliver later in the month if carrying the bond is profitable.
- **Wildcard Option:** The right to trade after the close, adding further delivery timing flexibility.

**Arbitrage Limits:**  
Arbitrage does not drive the Net Basis to zero because these options represent real risks to the long futures position that cannot be perfectly hedged. Additionally, balance sheet constraints (e.g., SLR requirements) and transaction costs prevent arbitrageurs from completely closing the gap. The **positive basis is the market price of uncertainty** regarding both *what* (which bond) and *when* delivery will occur.